In [1]:
import os


In [2]:
%pwd

'C:\\Users\\DIKSHANT PATEL\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\DIKSHANT PATEL\\Kidney-Disease-Classification'

In [5]:
import tensorflow as tf

In [26]:
import dagshub
dagshub.init(repo_owner='DikshantPatel2210', repo_name='KidneyDiseaseClassification-DLProject', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

[2025-04-12 23:12:56,535: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/DikshantPatel2210/KidneyDiseaseClassification-DLProject "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "DikshantPatel2210/KidneyDiseaseClassification-DLProject"

[2025-04-12 23:12:56,556: INFO: helpers: Initialized MLflow to track repo "DikshantPatel2210/KidneyDiseaseClassification-DLProject"]


Repository DikshantPatel2210/KidneyDiseaseClassification-DLProject initialized!

[2025-04-12 23:12:56,560: INFO: helpers: Repository DikshantPatel2210/KidneyDiseaseClassification-DLProject initialized!]


In [27]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [29]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [30]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_roots])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/DikshantPatel2210/KidneyDiseaseClassification-DLProject.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [35]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from mlflow.models import ModelSignature
from mlflow.types import DataType, Schema

In [42]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [43]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-04-12 23:23:52,751: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-12 23:23:52,751: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-12 23:23:52,767: INFO: common: created directory at: artifacts]
Found 3732 images belonging to 4 classes.
234/234 [==============================] - 35s 150ms/step - loss: 6.8587 - accuracy: 0.6152
[2025-04-12 23:24:28,993: INFO: common: json file saved at: scores.json]


2025/04/12 23:24:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\DIKSHA~1\AppData\Local\Temp\tmpegc2ry_h\model\data\model\assets
[2025-04-12 23:24:31,773: INFO: builder_impl: Assets written to: C:\Users\DIKSHA~1\AppData\Local\Temp\tmpegc2ry_h\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/04/12 23:24:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.


In [24]:
print(mlflow.get_tracking_uri())


https://dagshub.com/DikshantPatel2210/KidneyDiseaseClassification-DLProject.mlflow


In [25]:
mlflow.registered_models.list_registered_models()

AttributeError: module 'mlflow' has no attribute 'registered_models'